# Test Tensorflow-federated (TFF) library

## Test #4 : custom `ClientWork` process

In [1]:
import os
import collections
import nest_asyncio
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_federated as tff

tf.config.set_visible_devices([tf.config.list_physical_devices('GPU')[0]], 'GPU')

nest_asyncio.apply()

print('Tensorflow version : {}'.format(tf.__version__))
print('Tensorflow-federated version : {}'.format(tff.__version__))
print('# GPUs : {}'.format(len(tf.config.list_logical_devices('GPU'))))

tff.federated_computation(lambda: 'Hello, World!')()

2022-11-22 18:34:33.247841: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-22 18:34:33.343990: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-22 18:34:33.367985: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/etinfo/users2/qlanglois/.local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is requir

Tensorflow version : 2.10.0
Tensorflow-federated version : 0.39.0
# GPUs : 1


2022-11-22 18:34:45.601701: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-22 18:34:45.986275: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14783 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:17:00.0, compute capability: 7.5


b'Hello, World!'

In [2]:
emnist_train, emnist_valid = tff.simulation.datasets.emnist.load_data()
print('Dataset length :\n  Train length : {}\n  Valid length : {}'.format(
    len(emnist_train.client_ids), len(emnist_valid.client_ids)
))
print('Data signature : {}'.format(emnist_train.element_type_structure))

Dataset length :
  Train length : 3383
  Valid length : 3383
Data signature : OrderedDict([('label', TensorSpec(shape=(), dtype=tf.int32, name=None)), ('pixels', TensorSpec(shape=(28, 28), dtype=tf.float32, name=None))])


In [3]:
def preprocess(dataset, epochs = 10, batch_size = 32, shuffle_size = 1024, prefetch_size = 16):
    def batch_format_fn(data):
        return (
            tf.reshape(data['pixels'], [-1, 28 * 28]),
            tf.reshape(data['label'], [-1, 1])
        )
    
    return dataset.repeat(epochs).batch(batch_size).map(batch_format_fn).prefetch(prefetch_size)

def make_federated_data(client_data, ids = None, n = None):
    if ids is None: ids = client_data.client_ids[:n]
    return ids, [
        preprocess(client_data.create_tf_dataset_for_client(x)) for x in ids
    ]

train_ids, train_fed_data = make_federated_data(emnist_train, n = 100)
valid_ids, valid_fed_data = make_federated_data(emnist_valid, n = 25)
valid_data     = preprocess(emnist_valid.create_tf_dataset_from_all_clients(), epochs = 1, batch_size = 256)

print('# datasets : train {} - valid {}'.format(len(train_fed_data), len(valid_fed_data)))

# datasets : train 100 - valid 25


## Custom ClientWork process

In [4]:
def finalize_and_concat_metrics(finalizers, metrics_type):
    compiled_finalizers = collections.OrderedDict()
    for k, fn in finalizers.items():
        compiled_finalizers[k] = tff.tf_computation(
            fn, metrics_type[k]
        )
    compiled_finalizers['id'] = tff.tf_computation(lambda x: x, metrics_type['id'])
    
    finalized_spec = tf.nest.map_structure(
        lambda fn: tf.TensorSpec(shape = (None,), dtype = fn.type_signature.result.dtype), compiled_finalizers
    )

    @tff.tf_computation(metrics_type)
    def finalize(unfinalized_metrics):
        finalized = collections.OrderedDict()
        for name, value in unfinalized_metrics.items():
            finalized[name] = finalizers[name](value) if name in finalizers else value
        return finalized
    
    @tff.tf_computation(finalized_spec, metrics_type)
    def accumulate(accumulator, values):
        return tf.nest.map_structure(
            lambda x, y: tf.concat([x, tf.expand_dims(y, 0)], 0), accumulator, finalize(values)
        )

    @tff.tf_computation
    def get_init_state():
        return tf.nest.map_structure(
            lambda m: tf.zeros((1,), dtype = m.dtype), finalized_spec
        )

    @tff.federated_computation(tff.type_at_clients(metrics_type))
    def finalize_and_concat(metrics):
        return tff.federated_aggregate(
            metrics,
            get_init_state(),
            accumulate = accumulate,
            merge      = tff.tf_computation(lambda x, y: y),
            report     = tff.tf_computation(lambda x: tf.nest.map_structure(lambda v: v[1:], x))
        )
    
    return finalize_and_concat
    
@tf.function
def client_update(model, dataset, server_weights, client_optimizer):
    client_variables = tff.learning.models.ModelWeights.from_model(model)
    
    tf.nest.map_structure(
        lambda w, s_w: w.assign(s_w), client_variables, server_weights
    )
    
    num_examples = 0.
    for batch in dataset:
        with tf.GradientTape() as tape:
            outputs = model.forward_pass(batch)
        
        num_examples += tf.cast(outputs.num_examples, tf.float32)
        
        grads = tape.gradient(outputs.loss, client_variables.trainable)
        client_optimizer.apply_gradients(zip(grads, client_variables.trainable))
    
    update = tf.nest.map_structure(
        tf.subtract, server_weights.trainable, client_variables.trainable
    )
    
    metrics = model.report_local_unfinalized_metrics()
    
    return tff.learning.templates.ClientResult(
        update = update, update_weight = num_examples
    ), metrics

def build_client_work(model_fn, optimizer_fn, aggregator = finalize_and_concat_metrics):
    with tf.Graph().as_default():
        model = model_fn()
        metric_tensors = model.report_local_unfinalized_metrics()
        metric_tensors['id'] = tf.zeros((), dtype = tf.int32)
        metrics_type = tff.framework.type_from_tensors(metric_tensors)
        metrics_aggregator = aggregator(model.metric_finalizers(), metrics_type)
    
    data_type    = tff.SequenceType(model.input_spec)
    cid_type     = tff.SequenceType(tf.TensorSpec(shape = (), dtype = tf.int32))
    weights_type = tff.learning.models.weights_type_from_model(model)

    @tff.federated_computation
    def initialize_fn():
        return tff.federated_value((), tff.SERVER)
    
    @tff.tf_computation(weights_type, cid_type, data_type)
    def client_update_computation(model_weights, client_id, dataset):
        cid = client_id.reduce(tf.zeros((), dtype = tf.int32), lambda _, y: y)
        model     = model_fn()
        optimizer = optimizer_fn()
        return client_update(model, dataset, model_weights, optimizer) + (cid, )

    @tff.tf_computation
    def add_client_id(c_id, metrics):
        metrics['id'] = c_id
        return metrics
    
    @tff.federated_computation(
        initialize_fn.type_signature.result,
        tff.type_at_clients(weights_type),
        tff.type_at_clients((cid_type, data_type))
    )
    def next_fn(state, model_weights, client_dataset_with_id):
        client_ids, client_dataset = client_dataset_with_id
        client_result, metrics, client_ids = tff.federated_map(
            client_update_computation, (model_weights, client_ids, client_dataset)
        )
        
        metrics_with_id = tff.federated_map(add_client_id, (client_ids, metrics))
        metrics = metrics_aggregator(metrics_with_id)
        client_ids = tff.federated_map(tff.tf_computation(lambda x: x), client_ids)
        
        measurements = tff.federated_zip(collections.OrderedDict(
            train_metrics = metrics
        ))
        return tff.templates.MeasuredProcessOutput(state, client_result, measurements)
    
    return tff.learning.templates.ClientWorkProcess(
        initialize_fn, next_fn
    )

#client_work  = build_client_work(model_fn, client_optimizer_fn)

## Model + process initialization

In [5]:
def build_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape = (28 * 28, ), dtype = tf.int32),
        tf.keras.layers.Dense(32, activation = 'relu'),
        tf.keras.layers.Dense(10, activation = 'softmax')
    ], name = 'simple_mlp')
    model.build((None, 28 * 28))
    return model

def model_fn():
    model = build_model()
    return tff.learning.from_keras_model(
        model,
        input_spec = train_fed_data[0].element_spec,
        loss       = tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics    = [tf.keras.metrics.SparseCategoricalAccuracy()]
    )

server_optimizer_fn = lambda: tf.keras.optimizers.SGD(0.5)
client_optimizer_fn = lambda: tf.keras.optimizers.SGD(0.1)

In [6]:
@tff.tf_computation
def initial_model_weights_fn():
    return tff.learning.models.ModelWeights.from_model(model_fn())

weights_type = initial_model_weights_fn.type_signature.result
aggregator_factory = tff.aggregators.MeanFactory()

distributor  = tff.learning.templates.build_broadcast_process(weights_type)
client_work  = build_client_work(model_fn, client_optimizer_fn)
aggregator   = aggregator_factory.create(weights_type.trainable, tff.TensorType(tf.float32))
finalizer    = tff.learning.templates.build_apply_optimizer_finalizer(server_optimizer_fn, weights_type)

training_process = tff.learning.templates.compose_learning_process(
    initial_model_weights_fn,
    distributor,
    client_work,
    aggregator,
    finalizer
) 

print(distributor.next.type_signature.formatted_representation())
print()
print(client_work.next.type_signature.formatted_representation())
print()
print(aggregator.next.type_signature.formatted_representation())
print()
print(finalizer.next.type_signature.formatted_representation())
print()

print(training_process.initialize.type_signature.formatted_representation())
print(training_process.next.type_signature.formatted_representation())

(<
  state=<>@SERVER,
  value=<
    trainable=<
      float32[784,32],
      float32[32],
      float32[32,10],
      float32[10]
    >,
    non_trainable=<>
  >@SERVER
> -> <
  state=<>@SERVER,
  result=<
    trainable=<
      float32[784,32],
      float32[32],
      float32[32,10],
      float32[10]
    >,
    non_trainable=<>
  >@CLIENTS,
  measurements=<>@SERVER
>)

(<
  state=<>@SERVER,
  model_weights={<
    trainable=<
      float32[784,32],
      float32[32],
      float32[32,10],
      float32[10]
    >,
    non_trainable=<>
  >}@CLIENTS,
  client_dataset_with_id={<
    int32*,
    <
      float32[?,784],
      int32[?,1]
    >*
  >}@CLIENTS
> -> <
  state=<>@SERVER,
  result={<
    update=<
      float32[784,32],
      float32[32],
      float32[32,10],
      float32[10]
    >,
    update_weight=float32
  >}@CLIENTS,
  measurements=<
    train_metrics=<
      sparse_categorical_accuracy=float32[?],
      loss=float32[?],
      num_examples=int64[?],
      num_batches=int64[?

## Training and evaluation

In [7]:
state  = training_process.initialize()

sample_data = train_fed_data[:5]
sample_ids  = train_ids[:5]

seq_train_ids = [tf.data.Dataset.from_tensor_slices(np.array([train_ids.index(id_i)], dtype = np.int32)) for id_i in sample_ids]
result = training_process.next(state, list(zip(seq_train_ids, sample_data)))
print(result.metrics)

OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train_metrics', OrderedDict([('sparse_categorical_accuracy', array([0.10752688, 0.10952381, 0.0880734 , 0.13972603, 0.117     ],
      dtype=float32)), ('loss', array([2.3770826, 2.3408532, 2.3351161, 2.3583326, 2.343554 ],
      dtype=float32)), ('num_examples', array([ 930, 1050, 1090,  730, 1000])), ('num_batches', array([30, 33, 35, 23, 32])), ('id', array([0, 4, 1, 2, 3], dtype=int32))]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', ())])


In [8]:
#state  = training_process.initialize()

sample_data = train_fed_data[:1]
sample_ids  = train_ids[:1]

seq_train_ids = [tf.data.Dataset.from_tensor_slices(np.array([train_ids.index(id_i)], dtype = np.int32)) for id_i in sample_ids]
result = training_process.next(state, list(zip(seq_train_ids, sample_data)))
print(result.metrics)

OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train_metrics', OrderedDict([('sparse_categorical_accuracy', array([0.10752688], dtype=float32)), ('loss', array([2.3770826], dtype=float32)), ('num_examples', array([930])), ('num_batches', array([30])), ('id', array([0], dtype=int32))]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', ())])


In [9]:
state  = training_process.initialize()

seq_train_ids = [tf.data.Dataset.from_tensor_slices(np.array([train_ids.index(id_i)], dtype = np.int32)) for id_i in train_ids]
result = training_process.next(state, list(zip(seq_train_ids, train_fed_data)))
print(result)

2022-11-22 18:35:02.929575: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:35:02.935989: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:35:03.225229: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:35:03.430400: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:35:03.524371: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:35:03.714996: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:35:03.857446: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:35:03.928495: W tensorflow/core/data/root_dataset.cc:26

LearningProcessOutput(state=LearningAlgorithmState(global_model_weights=ModelWeights(trainable=[array([[-0.08805034,  0.07923926,  0.01819538, ..., -0.05306567,
         0.05572082, -0.01235208],
       [ 0.0493898 ,  0.05937357, -0.02925503, ...,  0.01768058,
        -0.07691772, -0.02774272],
       [ 0.00774906,  0.04109582,  0.07772142, ...,  0.0308284 ,
         0.04236679, -0.03112888],
       ...,
       [ 0.07290057, -0.0816852 , -0.00111948, ...,  0.05589662,
        -0.01372504,  0.00338438],
       [-0.03415132,  0.0375578 , -0.04986684, ..., -0.01873267,
        -0.02357909, -0.0423255 ],
       [-0.00164623,  0.04110616, -0.04004874, ..., -0.04505235,
        -0.07560211,  0.03686453]], dtype=float32), array([-0.00380583,  0.        , -0.00417167, -0.00346349, -0.00460942,
       -0.00554304, -0.00261627, -0.00284537, -0.00456945, -0.00082845,
       -0.00878817, -0.00102341, -0.00664818,  0.        ,  0.        ,
       -0.00458362, -0.00015689, -0.00055143, -0.00176137, 

In [10]:
print(result.metrics)

OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train_metrics', OrderedDict([('sparse_categorical_accuracy', array([0.12474227, 0.05555556, 0.0945055 , 0.11428571, 0.08301887,
       0.10091743, 0.1049505 , 0.08173077, 0.09417476, 0.12820514,
       0.13466667, 0.10103093, 0.07589286, 0.14285715, 0.11309524,
       0.13972603, 0.04385965, 0.07032967, 0.10204082, 0.09065421,
       0.08640777, 0.16533333, 0.11595745, 0.10952381, 0.05478261,
       0.10636364, 0.11752577, 0.11470588, 0.10093458, 0.12      ,
       0.09166667, 0.10315789, 0.11037736, 0.1204301 , 0.07523809,
       0.10104167, 0.10808081, 0.117     , 0.046875  , 0.08411215,
       0.12      , 0.0699115 , 0.1368421 , 0.0509434 , 0.07522935,
       0.09759036, 0.1       , 0.11648352, 0.073     , 0.07475728,
       0.2375    , 0.09339622, 0.15714286, 0.10943396, 0.11888889,
       0.10617284, 0.07764706, 0.10779221, 0.07238095, 0.12121212,
       0.11075269, 0.0880734 , 0.059375  , 0.05964912, 0.07816092,
   

In [11]:
epochs = 25

state  = training_process.initialize()

for i in range(epochs):
    indices = np.random.choice(len(train_fed_data), size = 50, replace = False)

    train_samp_ids = [train_ids[idx] for idx in indices]
    train_samples  = [train_fed_data[idx] for idx in indices]
    
    seq_train_ids = [tf.data.Dataset.from_tensor_slices(np.array([train_ids.index(id_i)], dtype = np.int32)) for id_i in train_samp_ids]
    result = training_process.next(state, list(zip(seq_train_ids, train_fed_data)))
    state  = result.state
    print(tf.nest.map_structure(
        lambda x: np.mean(x), result.metrics['client_work']['train_metrics']
    ))


2022-11-22 18:35:21.539343: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:35:21.561484: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:35:21.688096: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:35:22.236937: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:35:22.265346: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:35:22.308351: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:35:22.317699: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:35:22.362336: W tensorflow/core/data/root_dataset.cc:26

OrderedDict([('sparse_categorical_accuracy', 0.10567128), ('loss', 2.36472), ('num_examples', 930.8), ('num_batches', 29.64), ('id', 46.18)])


2022-11-22 18:35:29.362357: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:35:29.363642: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:35:29.363766: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:35:29.366677: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:35:29.387614: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:35:29.408770: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:35:29.411843: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:35:29.450358: W tensorflow/core/data/root_dataset.cc:26

OrderedDict([('sparse_categorical_accuracy', 0.119901255), ('loss', 2.2991133), ('num_examples', 930.8), ('num_batches', 29.64), ('id', 52.96)])


2022-11-22 18:35:32.038978: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:35:32.041995: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:35:32.087628: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:35:32.165825: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:35:32.205097: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:35:32.218777: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:35:32.218810: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:35:32.219345: W tensorflow/core/data/root_dataset.cc:26

...

OrderedDict([('sparse_categorical_accuracy', 0.12585007), ('loss', 2.2944746), ('num_examples', 930.8), ('num_batches', 29.64), ('id', 56.4)])


2022-11-22 18:36:31.184239: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:36:31.212105: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:36:31.236804: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:36:31.255986: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:36:31.436225: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:36:31.437955: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:36:31.443794: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:36:31.476293: W tensorflow/core/data/root_dataset.cc:26

OrderedDict([('sparse_categorical_accuracy', 0.1259104), ('loss', 2.2944083), ('num_examples', 930.8), ('num_batches', 29.64), ('id', 47.08)])


2022-11-22 18:36:34.398136: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:36:34.405596: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:36:34.443333: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:36:34.449953: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:36:34.451739: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:36:34.454433: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:36:34.456406: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-11-22 18:36:34.465363: W tensorflow/core/data/root_dataset.cc:26

OrderedDict([('sparse_categorical_accuracy', 0.12619293), ('loss', 2.2943647), ('num_examples', 930.8), ('num_batches', 29.64), ('id', 53.2)])


## Federated concatenation

In [12]:
input_types = (
    tf.TensorSpec(shape = (), dtype = tf.int32),
    tf.TensorSpec(shape = (), dtype = tf.float32)
)
agg_types = (
    tf.TensorSpec(shape = (None, ), dtype = tf.int32),
    tf.TensorSpec(shape = (None, ), dtype = tf.float32)
)

@tff.tf_computation(agg_types, input_types)
def accumulate(accumulator, accumuland):
    return tf.nest.map_structure(
        lambda x, y: tf.concat([x, tf.expand_dims(y, 0)], 0), accumulator, accumuland
    )

@tff.tf_computation
def merge(x, y):
    return x

@tff.tf_computation
def get_init_state():
    return (tf.fill((1,), -1), tf.zeros((1, ), dtype = tf.float32))

@tff.federated_computation(tff.type_at_clients(input_types))
def next_fn(values):
    agg_result = tff.federated_aggregate(
        values,
        get_init_state(),
        accumulate=accumulate,
        merge=merge,
        report=tff.tf_computation(lambda x: x)
    )    #return tff.federated_map(add_one, values)
    return agg_result #tff.federated_map(tff.tf_computation(lambda x: x[1:]), agg_result)

print(next_fn.type_signature.formatted_representation())

({<
  int32,
  float32
>}@CLIENTS -> <
  int32[?],
  float32[?]
>@SERVER)


In [13]:
for _ in range(5):
    print(next_fn([(i, float(i ** 2)) for i in range(3)]))

(array([-1,  1,  2,  0], dtype=int32), array([0., 1., 4., 0.], dtype=float32))
(array([-1,  0,  2,  1], dtype=int32), array([0., 0., 4., 1.], dtype=float32))
(array([-1,  0,  2,  1], dtype=int32), array([0., 0., 4., 1.], dtype=float32))
(array([-1,  1,  2,  0], dtype=int32), array([0., 1., 4., 0.], dtype=float32))
(array([-1,  0,  2,  1], dtype=int32), array([0., 0., 4., 1.], dtype=float32))


In [14]:
help(tff.TensorType)

Help on class TensorType in module tensorflow_federated.python.core.impl.types.computation_types:

class TensorType(Type)
 |  TensorType(*args, **kwargs)
 |  
 |  An implementation of `tff.Type` representing types of tensors in TFF.
 |  
 |  Method resolution order:
 |      TensorType
 |      Type
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Determines whether two type definitions are identical.
 |      
 |      Note that this notion of equality is stronger than equivalence. Two types
 |      with equivalent definitions may not be identical, e.g., if they represent
 |      templates with differently named type variables in their definitions.
 |      
 |      Args:
 |        other: The other type to compare against.
 |      
 |      Returns:
 |        `True` iff type definitions are syntatically identical (as defined above),
 |        or `False` otherwise.
 |      
 |      Raises:
 |        NotImplementedError: If not implemented in the der